- author: Vincent DAGOURY
- date: 2020-05-01
- email: vincent.dagoury@gmail.com
- github : [https://github.com/vincedgy](https://github.com/vincedgy)

# Analyse and transform docx file to JSON representation

The purpose of this notebook is to analyse the content of formated docx file.
This file is used for medical report.


In [1]:
%pip install python-docx lxml

Note: you may need to restart the kernel to use updated packages.


## Analyse the document

In [2]:
import docx
import re
from datetime import datetime
import uuid
import pandas as pd
import zipfile
from lxml import etree

## Util functions

In [3]:
def read_docx(docx_file, **kwargs):
    """Read tables as DataFrames from a Word document"""
    ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
    with zipfile.ZipFile(docx_file).open('word/document.xml') as f:
        root = etree.parse(f)
    for el in root.xpath('//w:tbl', namespaces=ns):
        el.tag = 'table'
    for el in root.xpath('//w:tr', namespaces=ns):
        el.tag = 'tr'
    for el in root.xpath('//w:tc', namespaces=ns):
        el.tag = 'td'
    return pd.read_html(etree.tostring(root), **kwargs)


def clean_token(token):
  """Cleans token"""
  t=str(token).strip()
  t=t.replace('\xa0','')
  t=t.replace('«','')
  t=t.replace('»','')
  t=t.replace(':','')
  return t

def read_tokens_from_docx(docx_file, **kwargs):
    """Read tokens of text from a Word document"""
    blacklist=['',':','&','-','.']
    ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
    with zipfile.ZipFile(docx_file).open('word/document.xml') as f:
        root = etree.parse(f)
    tokens=[]
    for el in root.xpath('//w:t', namespaces=ns):
      t=clean_token(el.text)
      if t != '' and t not in blacklist:
        tokens.append(t)    
    return tokens

## Construct 'analyse' result object

In [4]:
# Result dictionnary
analyse = {}

# Attributes
key_metadata='metadata'
key_data='data'
analyse[key_metadata]={}
analyse[key_data]={}

# Produces ids for this analyse
now = datetime.now()
analyse['datetime']=now.strftime("%Y%m%d%H%M%S")
analyse['timestamp']=str(now.timestamp())
analyse['id']=str(uuid.uuid4())

## Building document internal object

In [5]:
# Process the doc (with python-docx)
doc = docx.Document('file.docx')

# Process XML tokens in docx file
tokens=read_tokens_from_docx(docx_file='file.docx')
analyse[key_data]['title']=tokens[0]

# Also build tables to DataFrames
df2 = read_docx(docx_file='file.docx')

### Example with tokens

In [6]:
# Find a (the next) token based on the content of previsous one
nb_tokens=len(tokens)
for i in range(nb_tokens):
  if 'Date opératoire' in tokens[i]:
    print(tokens[i+1])

17/03/2020


## Document metadata

In [7]:
analyse[key_metadata]['title']=doc.core_properties.title
analyse[key_metadata]['author']=doc.core_properties.author
analyse[key_metadata]['created']=str(doc.core_properties.created)
analyse[key_metadata]['modified']=str(doc.core_properties.modified)
analyse[key_metadata]['last_modified_by']=str(doc.core_properties.last_modified_by)
analyse[key_metadata]['last_printed']=str(doc.core_properties.last_printed)
analyse[key_metadata]['revision']=doc.core_properties.revision
analyse[key_metadata]['subject']=doc.core_properties.subject
analyse[key_metadata]['identifier']=doc.core_properties.identifier
analyse[key_metadata]['comments']=doc.core_properties.comments
analyse[key_metadata]['category']=doc.core_properties.category
analyse[key_metadata]['keywords']=doc.core_properties.keywords
analyse[key_metadata]['content_status']=doc.core_properties.content_status
analyse[key_metadata]['language']=doc.core_properties.language
analyse[key_metadata]['version']=doc.core_properties.version
analyse[key_metadata]

{'title': 'Institut Cardiovasculaire Paris Sud',
 'author': 'Bloc HPJC',
 'created': '2020-04-29 18:40:00',
 'modified': '2020-04-29 19:31:00',
 'last_modified_by': 'Jérôme Lacotte',
 'last_printed': '2020-03-17 16:00:00',
 'revision': 42,
 'subject': '',
 'identifier': '',
 'comments': '',
 'category': '',
 'keywords': '',
 'content_status': '',
 'language': '',
 'version': ''}

### Catching specific paragraphs

In [8]:
all_paragraphs = doc.paragraphs
fullText = []
for paragraph in all_paragraphs:
  text = paragraph.text
  fullText.append(text)
  # 'Indication'
  if 'Indication' in text:
    analyse[key_data]['Indication'] = re.search(r'^Indication\s:\s(.+)$', text).group(1)
  # 'Remarque'
  if 'Remarque' in text:
    analyse[key_data]['Remarque'] = re.search(r'^Remarque\s:\s(.+)$', text).group(1)
  # 'Fermeture'
  if 'Fermeture' in text:
    analyse[key_data]['Fermeture'] = text   

#print('\n'.join(fullText))

## Work on DataFrame

### 'Main tab' part

In [9]:
df_tab1=df2[0]
analyse[key_data]['Nom'] = clean_token(df_tab1[1][0])
analyse[key_data]['Prénom'] = clean_token(df_tab1[3][0])
analyse[key_data]['Né(e) le'] = clean_token(df_tab1[1][1])
analyse[key_data]['Opérateur(s)'] = clean_token(df_tab1[1][2])
analyse[key_data]['Anesthésiste'] = clean_token(df_tab1[1][3])
analyse[key_data]['Anesthésie'] = clean_token(df_tab1[3][3])
#analyse[key_data]['Date opératoire'] = 'NaN'
analyse[key_data]['Début - Fin'] = clean_token(df_tab1[2][4])
analyse[key_data]['NIP'] = clean_token(df_tab1[5][1])
analyse[key_data]['Durée Rx (min)'] = clean_token(df_tab1[1][5])
analyse[key_data]['PDS (µGy.m²)'] = clean_token(df_tab1[3][5])

# With tokens
for i in range(len(tokens)):
  if 'Date opératoire' in tokens[i]:
    analyse[key_data]['Date opératoire']=clean_token(tokens[i+1])
  if 'Scopie' in tokens[i]:
    analyse[key_data]['Scopie']=clean_token(tokens[i+1])
analyse[key_data]

{'title': 'CHANGEMENT DE STIMULATEUR CARDIAQUE (DEKA 001)',
 'Indication': 'Usure batterie. Dysfonction sinusale.',
 'Remarque': 'intervention en urgence',
 'Fermeture': 'Fermeture en 3 plans. Sous peau = Vicryl ® 2-0\xa0; peau =   ',
 'Nom': 'NOMANONYME',
 'Prénom': 'PRENOMANONYME',
 'Né(e) le': '12/04/1928',
 'Opérateur(s)': 'Dr(s) Mina AIT SAID & Pascale CHEMALY',
 'Anesthésiste': 'Dr Laurent TELLIER',
 'Anesthésie': 'locale et neuroleptanalgésie',
 'Début - Fin': '15h30-16h',
 'NIP': '202002582.0',
 'Durée Rx (min)': '2',
 'PDS (µGy.m²)': '3',
 'Date opératoire': '17/03/2020',
 'Scopie': "Siemens CIOS ALPHA - mise en service le 10/03/2015 - numéro d'installation L035874"}

### 'Symptomes' part

In [10]:
df_tab2=df2[1]
analyse[key_data]['symptomes']={}
for i in range(4):
  analyse[key_data]['symptomes'][df_tab2[i][0]] = clean_token(df_tab2[i][1])
analyse[key_data]['symptomes']

{'Cardiopathie': 'ischémique',
 'Cardiomyopathie': 'non',
 'Autre': 'non',
 'Canalopathie et autre': 'non'}

### 'Protocole' part

In [11]:
df_tab3=df2[2]
analyse[key_data]['protocole']={}
analyse[key_data]['protocole'][df_tab3[0][0]] = clean_token(df_tab3[0][1])
analyse[key_data]['protocole'][df_tab3[1][0]] = clean_token(df_tab3[1][1])
analyse[key_data]['protocole'][df_tab3[1][0] + "-2"] = clean_token(df_tab3[2][1])
analyse[key_data]['protocole'][df_tab3[2][0]] = clean_token(df_tab3[3][1])
analyse[key_data]['protocole'][df_tab3[2][0] + "-2"] = clean_token(df_tab3[4][1])
analyse[key_data]['protocole'][df_tab3[3][0]] = clean_token(df_tab3[3][1])
#analyse[key_data]['protocole'][df_tab3[3][0]] = 'NaN'


# NYHA and FE with tokens
for i in range(len(tokens)):
  if 'NYHA' in tokens[i]:
    analyse[key_data]['protocole']['NYHA']=clean_token(tokens[i+9])
  if 'FE' in tokens[i]:
    analyse[key_data]['protocole']['FE']=clean_token(tokens[i+9])
    
analyse[key_data]['protocole']

{'Antibradycardique': 'BAV III',
 'Resynchronisation': 'non',
 'Resynchronisation-2': 'QRS  110 ms',
 'Antitachycardique': 'non',
 'Antitachycardique-2': 'pas de SVP',
 'NYHA': 'I',
 'FE': '50'}

In [12]:
print(tokens)

['CHANGEMENT DE STIMULATEUR CARDIAQUE (DEKA 001)', 'Nom', 'NOMANONYME', 'Prénom', 'PRENOMANONYME', 'Né(é) le', '12/04/1928', 'NIP', '202002582', 'Opérateur(s)', 'Dr(s)', 'Mina AIT SAID', 'Pascale CHEMALY', 'Anesthésiste ', 'Dr', 'Laurent TELLIER', 'Anesthésie', 'locale et neuroleptanalgésie', 'Date opératoire', '17/03/2020', 'Début', 'Fin', '15h30-16h', 'Durée', 'Rx', '(min)', '2', 'PDS', '(µGy.m²)', '3', 'Scopie', "Siemens CIOS ALPHA - mise en service le 10/03/2015 - numéro d'installation L035874", 'Indication', 'Usure batterie. Dysfonction sinusale', 'Symptômes', 'asthénie', 'Cardiopathie', 'Cardiomyopathie', 'Autre', 'Canalopathie et autre', 'ischémique', 'non', 'non', 'non', 'Antibradycardique', 'Resynchronisation', 'Antitachycardique', 'NYHA', 'FE', 'BAV III', 'non', 'QRS', '110', 'ms', 'non', 'pas de SVP', 'I', '50', '%', 'Remarque', 'intervention en urgence', 'Matériel', 'Sondes', 'Position', 'OD', 'VD', 'VG', 'Etat', 'implantée', 'implantée', 'implantée', 'Localisation', 'endoc

### 'Sondes' part

In [13]:
df_tab4=df2[3]
analyse[key_data]['materiel']={}
analyse[key_data]['materiel']['sondes']=[]

for i in range(3):
  materiel={}
  for j in range(11):
    materiel[clean_token(df_tab4[0][j])]=clean_token(df_tab4[i+1][j])
  for k in range(len(tokens)):
    if 'Date d’implantation' in tokens[k]:
      materiel['Date d’implantation']=tokens[k+i+1]
  analyse[key_data]['materiel']['sondes'].append(materiel)
  
analyse[key_data]['materiel']['sondes']

[{'Position': 'OD',
  'Etat': 'implantée',
  'Localisation': 'endocavitaire',
  'Date d’implantation': '22/03/2011',
  'Abord veineux': 'céphalique',
  'Marque / Modèle': 'Medtronic 5076-52 cm',
  'N° de série': 'PJN2484323',
  'IRM conditional': 'nan',
  'Seuil de stim. à 0.5 ms': '1',
  'Détection': '2',
  'Impédance à 5 V': '560'},
 {'Position': 'VD',
  'Etat': 'implantée',
  'Localisation': 'endocavitaire',
  'Date d’implantation': '22/03/2011',
  'Abord veineux': 'céphalique',
  'Marque / Modèle': 'Medtronic 5076-58 cm',
  'N° de série': 'PJN2477911',
  'IRM conditional': 'nan',
  'Seuil de stim. à 0.5 ms': '0.8',
  'Détection': '15',
  'Impédance à 5 V': '450'},
 {'Position': 'VG',
  'Etat': 'implantée',
  'Localisation': 'endocavitaire',
  'Date d’implantation': '22/03/2011',
  'Abord veineux': 'nan',
  'Marque / Modèle': 'Biotronik Solia S60',
  'N° de série': 'PJN2477733',
  'IRM conditional': 'nan',
  'Seuil de stim. à 0.5 ms': '1.2',
  'Détection': '10',
  'Impédance à 5 V':

### 'Boitier' part

In [14]:
df_tab5=df2[4]

analyse[key_data]['materiel']['boitier']={}
analyse[key_data]['materiel']['boitier'][df_tab5[0][0]]=df_tab5[1][0]
#analyse[key_data]['materiel']['boitier'][df_tab5[2][0]]='Nan'
#analyse[key_data]['materiel']['boitier'][df_tab5[0][1]]='Nan'
#analyse[key_data]['materiel']['boitier'][df_tab5[1][1]]='Nan'
analyse[key_data]['materiel']['boitier'][df_tab5[3][0]]=df_tab5[4][0]
analyse[key_data]['materiel']['boitier'][df_tab5[2][1]]=df_tab5[3][1]
#analyse[key_data]['materiel']['boitier'][df_tab5[4][2]]='Nan'


# Add tokens for 'boitier'
found_boitier=False

for i in range(len(tokens)):

  # This starts 'boitier' section
  if 'Boîtier' in tokens[i]:
    analyse[key_data]['materiel']['boitier']['Etat']=tokens[i+4]
    found_boitier=True
  if found_boitier and 'Position' in tokens[i]:
    analyse[key_data]['materiel']['boitier']['Position']=tokens[i+1]
  if found_boitier and 'Localisation' in tokens[i]:
    analyse[key_data]['materiel']['boitier']['Localisation']=tokens[i+1]
  if found_boitier and 'IRM' in tokens[i] and 'conditional' in tokens[i+1]:
    analyse[key_data]['materiel']['boitier']['IRM conditional']=tokens[i+2]
    found_boitier=False 
    # This ends 'boitier' section
    

analyse[key_data]['materiel']['boitier']

{'Type': 'Stimulateur',
 'Marque / Modèle': 'Biotronik Edora 8 DR T',
 'N° de série': '69578914',
 'Etat': 'implanté',
 'Position': 'gauche',
 'Localisation': 'pré-pectoral',
 'IRM conditional': 'oui'}

### 'Ancien boitier' part

In [15]:
df_tab6=df2[5]

analyse[key_data]['materiel']['ancien_boitier']={}
analyse[key_data]['materiel']['ancien_boitier'][df_tab6[0][0]]=df_tab6[1][0]
#analyse[key_data]['materiel']['ancien_boitier'][df_tab5[2][0]]='Nan'
#analyse[key_data]['materiel']['ancien_boitier'][df_tab6[0][1]]='Nan'
#analyse[key_data]['materiel']['ancien_boitier'][df_tab6[1][1]]='Nan'
analyse[key_data]['materiel']['ancien_boitier'][df_tab6[3][0]]=df_tab6[4][0]
analyse[key_data]['materiel']['ancien_boitier'][df_tab6[2][1]]=df_tab6[3][1]

# Add tokens for 'ancien_boitier'
found_ancien_boitier=False
for i in range(len(tokens)):
  if 'Ancien boitier' in tokens[i]:
    analyse[key_data]['materiel']['ancien_boitier']['Etat']=tokens[i+4]
    found_ancien_boitier=True
  if found_ancien_boitier and 'Position' in tokens[i]:
    analyse[key_data]['materiel']['ancien_boitier']['Position']=tokens[i+1]
  if found_ancien_boitier and 'Localisation' in tokens[i]:
    analyse[key_data]['materiel']['ancien_boitier']['Localisation']=tokens[i+1]
    found_ancien_boitier=False
    
analyse[key_data]['materiel']['ancien_boitier']

{'Type': 'Stimulateur',
 'Marque / Modèle': 'St Jude Accent DR',
 'N° de série': '4250261',
 'Etat': 'explanté',
 'Position': 'gauche',
 'Localisation': 'pré-pectoral'}

## Final 'analyse' object

In [16]:
analyse

{'metadata': {'title': 'Institut Cardiovasculaire Paris Sud',
  'author': 'Bloc HPJC',
  'created': '2020-04-29 18:40:00',
  'modified': '2020-04-29 19:31:00',
  'last_modified_by': 'Jérôme Lacotte',
  'last_printed': '2020-03-17 16:00:00',
  'revision': 42,
  'subject': '',
  'identifier': '',
  'comments': '',
  'category': '',
  'keywords': '',
  'content_status': '',
  'language': '',
  'version': ''},
 'data': {'title': 'CHANGEMENT DE STIMULATEUR CARDIAQUE (DEKA 001)',
  'Indication': 'Usure batterie. Dysfonction sinusale.',
  'Remarque': 'intervention en urgence',
  'Fermeture': 'Fermeture en 3 plans. Sous peau = Vicryl ® 2-0\xa0; peau =   ',
  'Nom': 'NOMANONYME',
  'Prénom': 'PRENOMANONYME',
  'Né(e) le': '12/04/1928',
  'Opérateur(s)': 'Dr(s) Mina AIT SAID & Pascale CHEMALY',
  'Anesthésiste': 'Dr Laurent TELLIER',
  'Anesthésie': 'locale et neuroleptanalgésie',
  'Début - Fin': '15h30-16h',
  'NIP': '202002582.0',
  'Durée Rx (min)': '2',
  'PDS (µGy.m²)': '3',
  'Date opérat

## Build a JSON based representation

In [17]:
import json
i=json.dumps(analyse, sort_keys=True, indent=4)
print(i)

{
    "data": {
        "Anesth\u00e9sie": "locale et neuroleptanalg\u00e9sie",
        "Anesth\u00e9siste": "Dr Laurent TELLIER",
        "Date op\u00e9ratoire": "17/03/2020",
        "Dur\u00e9e Rx (min)": "2",
        "D\u00e9but - Fin": "15h30-16h",
        "Fermeture": "Fermeture en 3 plans. Sous peau = Vicryl \u00ae 2-0\u00a0; peau =   ",
        "Indication": "Usure batterie. Dysfonction sinusale.",
        "NIP": "202002582.0",
        "Nom": "NOMANONYME",
        "N\u00e9(e) le": "12/04/1928",
        "Op\u00e9rateur(s)": "Dr(s) Mina AIT SAID & Pascale CHEMALY",
        "PDS (\u00b5Gy.m\u00b2)": "3",
        "Pr\u00e9nom": "PRENOMANONYME",
        "Remarque": "intervention en urgence",
        "Scopie": "Siemens CIOS ALPHA - mise en service le 10/03/2015 - num\u00e9ro d'installation L035874",
        "materiel": {
            "ancien_boitier": {
                "Etat": "explant\u00e9",
                "Localisation": "pr\u00e9-pectoral",
                "Marque / Mod\u00e8le":

## Save to an output file

In [18]:
with open('file.json', 'w') as w:
  w.write(json.dumps(analyse, sort_keys=True, indent=4))